# 쿠팡 데이터 크롤링

___

## Global functions, parameters

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup

In [2]:
def getURL(seed, params=None, select='a', num_retries=2):
    html = requests.get(seed, params=params, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getURL(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    links = dom.select(select)
    
    return ['http://www.coupang.com' + link['href'] for link in links if link.has_attr('href')]

In [3]:
def getItemNum(url, params, num_retries=2):
    params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())
    html = requests.get(url, params=params_str, headers=headers)
    print(html.url)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getItemNum(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    result = dom.select('em.newcx-product-total-count')
    
    return result

In [4]:
headers = {
    "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
    "accept-encoding":"false"
}

___

### 과자
1. url
2. img
3. brand
4. title: 제품명(있는 그대로)
5. product: 보여줄 제품명(브랜드 + 연령대 + 형태)
6. price: 가격(int)
7. rating: 평점
8. replyCount: 상품평 갯수
9. reply: 20개

In [5]:
month_filter_snack = {
    '3개월 이하': '221915%23attr_11932%2413989%40DEFAULT',
    '3개월 이상 7개월 이하': '221915%23attr_11932%2413990%40DEFAULT',
    '7개월 이상 11개월 이하': '221915%23attr_11932%2413991%40DEFAULT',
    '11개월 이상 23개월 이하': '221915%23attr_11932%2413992%40DEFAULT',
    '23개월 이상 35개월 이하': '221915%23attr_11932%2413993%40DEFAULT',
    '35개월 이상': '221915%23attr_11932%2413994%40DEFAULT'
}

In [6]:
monthList_snack = ['3개월 이하', '3개월 이상 7개월 이하', '7개월 이상 11개월 이하', '11개월 이상 23개월 이하', '23개월 이상 35개월 이하', '35개월 이상']

In [7]:
params_snack = {
    "listSize": "60",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "221915",
    "rating": "0",
}

In [8]:
seed_snack = 'http://www.coupang.com/np/categories/222015'

In [9]:
result_snack = {
    monthList_snack[0]: None,
    monthList_snack[1]: None,
    monthList_snack[2]: None,
    monthList_snack[3]: None,
    monthList_snack[4]: None,
    monthList_snack[5]: None
}

for i in range(6):
    currentpage = 1
    params_snack['filter'] = month_filter_snack[monthList_snack[i]]
    temp = []
    
    while True:
        params_snack['page'] = str(currentpage)
        print(params_snack['filter'])
        if len(getItemNum(seed_snack, params_snack)) == 0:
            break
        print(params_snack['page'])
        temp.extend(getURL(seed_snack, params_snack, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_snack[monthList_snack[i]] = temp

221915%23attr_11932%2413989%40DEFAULT
http://www.coupang.com/np/categories/222015?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221915%23attr_11932%2413989%40DEFAULT&component=221915&rating=0
1
221915%23attr_11932%2413989%40DEFAULT
http://www.coupang.com/np/categories/222015?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221915%23attr_11932%2413989%40DEFAULT&component=221915&rating=0
2
221915%23attr_11932%2413989%40DEFAULT
http://www.coupang.com/np/categories/222015?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221915%23attr_11932%2413989%40DEFAULT&component=221915&rating=0
3
221915%23attr_11932%2413989%40DEFAULT
http://www.coupan

In [10]:
for (k, v) in result_snack.items():
    print(k, len(v))

3개월 이하 180
3개월 이상 7개월 이하 420
7개월 이상 11개월 이하 300
11개월 이상 23개월 이하 180
23개월 이상 35개월 이하 60
35개월 이상 60


In [11]:
snackList = []

In [12]:
def getInfo_snack(url, monthInfo, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_snack(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0:
        brand = dom.select('a.prod-brand-name')[0].get_text()
        name = dom.select('h2.prod-buy-header__title')[0].get_text()
        price = int(dom.select('span.total-price strong')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        url = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        snackList.append(tuple([brand, name, price, url, img, monthInfo]))
    
    return None

In [13]:
for i in range(6):
    for url in result_snack[monthList_snack[i]]:
        getInfo_snack(url, monthList_snack[i])

In [14]:
len(snackList)

1185

___

### 분유
1. url
2. img
3. brand
4. origin: 원산지(국내 / 해외)
5. title: 제품명(있는 그대로)
6. product: 보여줄 제품명(브랜드 + 연령대 + 형태)
7. stage: 분유 단계(int)
8. type: 분말 / 액상
9. price: 가격(int)
10. rating: 평점
11. replyCount: 상품평 갯수
12. reply: 20개

In [15]:
month_filter_milkPowder = {
    '3개월 이하': '221895%23attr_11932%2413989%40DEFAULT',
    '3개월 이상 7개월 이하': '221895%23attr_11932%2413990%40DEFAULT',
    '7개월 이상 11개월 이하': '221895%23attr_11932%2413991%40DEFAULT',
    '11개월 이상 23개월 이하': '221895%23attr_11932%2413992%40DEFAULT',
    '23개월 이상 35개월 이하': '221895%23attr_11932%2413993%40DEFAULT',
    '35개월 이상': '221895%23attr_11932%2413994%40DEFAULT'
}

In [16]:
monthList_milkPowder = ['3개월 이하', '3개월 이상 7개월 이하', '7개월 이상 11개월 이하', '11개월 이상 23개월 이하', '23개월 이상 35개월 이하', '35개월 이상']

In [17]:
params_milkPowder = {
    "listSize": "60",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "221895",
    "rating": "0",
}

In [18]:
seed_milkPowder = 'http://www.coupang.com/np/categories/221995'

In [19]:
result_milkPowder = {
    monthList_milkPowder[0]: None,
    monthList_milkPowder[1]: None,
    monthList_milkPowder[2]: None,
    monthList_milkPowder[3]: None,
    monthList_milkPowder[4]: None,
    monthList_milkPowder[5]: None
}

for i in range(6):
    currentpage = 1
    params_milkPowder['filter'] = month_filter_milkPowder[monthList_milkPowder[i]]
    temp = []
    
    while True:
        params_milkPowder['page'] = str(currentpage)
        print(params_milkPowder['filter'])
        if len(getItemNum(seed_milkPowder, params_milkPowder)) == 0:
            break
        print(params_milkPowder['page'])
        temp.extend(getURL(seed_milkPowder, params_milkPowder, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_milkPowder[monthList_milkPowder[i]] = temp

221895%23attr_11932%2413989%40DEFAULT
http://www.coupang.com/np/categories/221995?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221895%23attr_11932%2413989%40DEFAULT&component=221895&rating=0
1
221895%23attr_11932%2413989%40DEFAULT
http://www.coupang.com/np/categories/221995?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221895%23attr_11932%2413989%40DEFAULT&component=221895&rating=0
2
221895%23attr_11932%2413989%40DEFAULT
http://www.coupang.com/np/categories/221995?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221895%23attr_11932%2413989%40DEFAULT&component=221895&rating=0
3
221895%23attr_11932%2413989%40DEFAULT
http://www.coupan

http://www.coupang.com/np/categories/221995?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=6&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221895%23attr_11932%2413992%40DEFAULT&component=221895&rating=0
221895%23attr_11932%2413993%40DEFAULT
http://www.coupang.com/np/categories/221995?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221895%23attr_11932%2413993%40DEFAULT&component=221895&rating=0
1
221895%23attr_11932%2413993%40DEFAULT
http://www.coupang.com/np/categories/221995?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=221895%23attr_11932%2413993%40DEFAULT&component=221895&rating=0
221895%23attr_11932%2413994%40DEFAULT
http://www.coupang.com/np/categories/221995?listSize=60&bra

In [20]:
for (k, v) in result_milkPowder.items():
    print(k, len(v))

3개월 이하 540
3개월 이상 7개월 이하 480
7개월 이상 11개월 이하 60
11개월 이상 23개월 이하 300
23개월 이상 35개월 이하 60
35개월 이상 60


In [21]:
milkPowderList = []

In [22]:
def getInfo_milkPowder(url, monthInfo, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_milkPowder(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0:
        brand = dom.select('a.prod-brand-name')[0].get_text()
        name = dom.select('h2.prod-buy-header__title')[0].get_text()
        price = int(dom.select('span.total-price strong')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        url = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        milkPowderList.append(tuple([brand, name, price, url, img, monthInfo]))
    
    return None

In [23]:
for i in range(6):
    for url in result_milkPowder[monthList_milkPowder[i]]:
        getInfo_milkPowder(url, monthList_milkPowder[i])

In [24]:
len(milkPowderList)

1500

___

### 의류
1. 제품명
2. 성별
3. 가격
4. url
5. img
6. 연령대

In [25]:
gender_list = ['여아', '남아', '남녀공용']

In [26]:
month_filter_clothes_girl = {
    '0~3개월': '223745%23attr_11872%24150%40DEFAULT',
    '3~6개월': '223745%23attr_11872%24151%40DEFAULT',
    '6~9개월': '223745%23attr_11872%24152%40DEFAULT',
    '9~12개월': '223745%23attr_11872%24153%40DEFAULT',
    '12~18개월': '223745%23attr_11872%24154%40DEFAULT',
    '18~24개월': '223745%23attr_11872%24155%40DEFAULT'
}

In [27]:
month_filter_clothes_boy = {
    '0~3개월': '223746%23attr_11872%24150%40DEFAULT',
    '3~6개월': '223746%23attr_11872%24151%40DEFAULT',
    '6~9개월': '223746%23attr_11872%24152%40DEFAULT',
    '9~12개월': '223746%23attr_11872%24153%40DEFAULT',
    '12~18개월': '223746%23attr_11872%24154%40DEFAULT',
    '18~24개월': '223746%23attr_11872%24155%40DEFAULT'
}

In [28]:
month_filter_clothes_uni = {
    '0~3개월': '223747%23attr_11872%24150%40DEFAULT',
    '3~6개월': '223747%23attr_11872%24151%40DEFAULT',
    '6~9개월': '223747%23attr_11872%24152%40DEFAULT',
    '9~12개월': '223747%23attr_11872%24153%40DEFAULT',
    '12~18개월': '223747%23attr_11872%24154%40DEFAULT',
    '18~24개월': '223747%23attr_11872%24155%40DEFAULT'
}

In [29]:
monthList_clothes = ['0~3개월', '3~6개월', '6~9개월', '9~12개월', '12~18개월', '18~24개월']

In [30]:
params_clothes_girl = {
    "listSize": "60",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "223745",
    "rating": "0",
}

In [31]:
params_clothes_boy = {
    "listSize": "60",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "223746",
    "rating": "0",
}

In [32]:
params_clothes_uni = {
    "listSize": "60",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "223747",
    "rating": "0",
}

In [33]:
seed_clothes_girl = 'http://www.coupang.com/np/categories/223845'
seed_clothes_boy = 'http://www.coupang.com/np/categories/223846'
seed_clothes_uni = 'http://www.coupang.com/np/categories/223847'

In [34]:
result_clothes_girl = {
    monthList_clothes[0]: None,
    monthList_clothes[1]: None,
    monthList_clothes[2]: None,
    monthList_clothes[3]: None,
    monthList_clothes[4]: None,
    monthList_clothes[5]: None
}

for i in range(6):
    currentpage = 1
    params_clothes_girl['filter'] = month_filter_clothes_girl[monthList_clothes[i]]
    temp = []
    
    while True:
        params_clothes_girl['page'] = str(currentpage)
        print(params_clothes_girl['filter'])
        if len(getItemNum(seed_clothes_girl, params_clothes_girl)) == 0:
            break
        print(params_clothes_girl['page'])
        temp.extend(getURL(seed_clothes_girl, params_clothes_girl, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_clothes_girl[monthList_clothes[i]] = temp

223745%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24150%40DEFAULT&component=223745&rating=0
1
223745%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24150%40DEFAULT&component=223745&rating=0
2
223745%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24150%40DEFAULT&component=223745&rating=0
3
223745%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categ

9
223745%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=10&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24151%40DEFAULT&component=223745&rating=0
10
223745%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24151%40DEFAULT&component=223745&rating=0
11
223745%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24151%40DEFAULT&component=223745&rating=0
12
223745%23attr_11872%24151%40DEFAULT
http://www.coupang.com/

223745%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24153%40DEFAULT&component=223745&rating=0
1
223745%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24153%40DEFAULT&component=223745&rating=0
2
223745%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24153%40DEFAULT&component=223745&rating=0
3
223745%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categ

9
223745%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=10&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24154%40DEFAULT&component=223745&rating=0
10
223745%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24154%40DEFAULT&component=223745&rating=0
11
223745%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223845?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223745%23attr_11872%24154%40DEFAULT&component=223745&rating=0
12
223745%23attr_11872%24154%40DEFAULT
http://www.coupang.com/

In [35]:
for (k, v) in result_clothes_girl.items():
    print(k, len(v))

0~3개월 1020
3~6개월 1020
6~9개월 1020
9~12개월 1020
12~18개월 1020
18~24개월 1020


In [36]:
result_clothes_boy = {
    monthList_clothes[0]: None,
    monthList_clothes[1]: None,
    monthList_clothes[2]: None,
    monthList_clothes[3]: None,
    monthList_clothes[4]: None,
    monthList_clothes[5]: None
}

for i in range(6):
    currentpage = 1
    params_clothes_boy['filter'] = month_filter_clothes_boy[monthList_clothes[i]]
    temp = []
    
    while True:
        params_clothes_boy['page'] = str(currentpage)
        print(params_clothes_boy['filter'])
        if len(getItemNum(seed_clothes_boy, params_clothes_boy)) == 0:
            break
        print(params_clothes_boy['page'])
        temp.extend(getURL(seed_clothes_boy, params_clothes_boy, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_clothes_boy[monthList_clothes[i]] = temp

223746%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24150%40DEFAULT&component=223746&rating=0
1
223746%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24150%40DEFAULT&component=223746&rating=0
2
223746%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24150%40DEFAULT&component=223746&rating=0
3
223746%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categ

9
223746%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=10&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24151%40DEFAULT&component=223746&rating=0
10
223746%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24151%40DEFAULT&component=223746&rating=0
11
223746%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24151%40DEFAULT&component=223746&rating=0
12
223746%23attr_11872%24151%40DEFAULT
http://www.coupang.com/

223746%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24153%40DEFAULT&component=223746&rating=0
1
223746%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24153%40DEFAULT&component=223746&rating=0
2
223746%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24153%40DEFAULT&component=223746&rating=0
3
223746%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categ

9
223746%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=10&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24154%40DEFAULT&component=223746&rating=0
10
223746%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24154%40DEFAULT&component=223746&rating=0
11
223746%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223846?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223746%23attr_11872%24154%40DEFAULT&component=223746&rating=0
12
223746%23attr_11872%24154%40DEFAULT
http://www.coupang.com/

In [37]:
for (k, v) in result_clothes_boy.items():
    print(k, len(v))

0~3개월 1020
3~6개월 1020
6~9개월 1020
9~12개월 1020
12~18개월 1020
18~24개월 1020


In [38]:
result_clothes_uni = {
    monthList_clothes[0]: None,
    monthList_clothes[1]: None,
    monthList_clothes[2]: None,
    monthList_clothes[3]: None,
    monthList_clothes[4]: None,
    monthList_clothes[5]: None
}

for i in range(6):
    currentpage = 1
    params_clothes_uni['filter'] = month_filter_clothes_uni[monthList_clothes[i]]
    temp = []
    
    while True:
        params_clothes_uni['page'] = str(currentpage)
        print(params_clothes_uni['filter'])
        if len(getItemNum(seed_clothes_uni, params_clothes_uni)) == 0:
            break
        print(params_clothes_uni['page'])
        temp.extend(getURL(seed_clothes_uni, params_clothes_uni, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_clothes_uni[monthList_clothes[i]] = temp

223747%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24150%40DEFAULT&component=223747&rating=0
1
223747%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24150%40DEFAULT&component=223747&rating=0
2
223747%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24150%40DEFAULT&component=223747&rating=0
3
223747%23attr_11872%24150%40DEFAULT
http://www.coupang.com/np/categ

9
223747%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=10&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24151%40DEFAULT&component=223747&rating=0
10
223747%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24151%40DEFAULT&component=223747&rating=0
11
223747%23attr_11872%24151%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24151%40DEFAULT&component=223747&rating=0
12
223747%23attr_11872%24151%40DEFAULT
http://www.coupang.com/

223747%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24153%40DEFAULT&component=223747&rating=0
1
223747%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24153%40DEFAULT&component=223747&rating=0
2
223747%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24153%40DEFAULT&component=223747&rating=0
3
223747%23attr_11872%24153%40DEFAULT
http://www.coupang.com/np/categ

9
223747%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=10&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24154%40DEFAULT&component=223747&rating=0
10
223747%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24154%40DEFAULT&component=223747&rating=0
11
223747%23attr_11872%24154%40DEFAULT
http://www.coupang.com/np/categories/223847?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=223747%23attr_11872%24154%40DEFAULT&component=223747&rating=0
12
223747%23attr_11872%24154%40DEFAULT
http://www.coupang.com/

In [39]:
for (k, v) in result_clothes_uni.items():
    print(k, len(v))

0~3개월 1020
3~6개월 1020
6~9개월 1020
9~12개월 1020
12~18개월 1020
18~24개월 1020


In [40]:
clothesList = []

In [41]:
def getInfo_clothes(url, monthInfo, gender, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_clothes(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0 and len(dom.select('span.total-price strong')) > 0:
        name = dom.select('h2.prod-buy-header__title')[0].get_text()
        price = int(dom.select('span.total-price strong')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        url = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        clothesList.append(tuple([name, gender, price, url, img, monthInfo]))
    
    return None

In [42]:
def getall_clothes_boy():
    for i in range(6):
        for url in result_clothes_boy[monthList_clothes[i]]:
            getInfo_clothes(url, monthList_clothes[i], gender_list[1])
    
    print('Done-boy')
    return None

def getall_clothes_girl():
    for i in range(6):
        for url in result_clothes_girl[monthList_clothes[i]]:
            getInfo_clothes(url, monthList_clothes[i], gender_list[0])
    
    print('Done-girl')
    return None

def getall_clothes_uni():
    for i in range(6):
        for url in result_clothes_uni[monthList_clothes[i]]:
            getInfo_clothes(url, monthList_clothes[i], gender_list[2])
    
    print('Done-uni')
    return None

In [43]:
import threading
 
thread_boy = threading.Thread(target=getall_clothes_boy)
thread_girl = threading.Thread(target=getall_clothes_girl)
thread_uni = threading.Thread(target=getall_clothes_uni)

thread_boy.start()
thread_girl.start()
thread_uni.start()

In [62]:
len(clothesList)

17154

Done-girl
Done-boy
Done-uni


In [68]:
len(clothesList)

18293

___

### 인형, 장난감
1. 제품명
2. 성별
3. 가격
4. url
5. img
6. 연령대

In [46]:
month_filter_toy_boy = {
    '24개월 이하': '1%23attr_10860%242293%7C332717%23attr_11932%2415622%40DEFAULT',
    '2세 이상~5세 이하': '1%23attr_10860%242293%7C332717%23attr_11932%2415623%40DEFAULT',
    '5세 이상~8세 이하': '1%23attr_10860%242293%7C332717%23attr_11932%2415624%40DEFAULT',
    '8세 이상~13세 이하': '1%23attr_10860%242293%7C332717%23attr_11932%2415625%40DEFAULT',
    '13세 이상': '1%23attr_10860%242293%7C332717%23attr_11932%2415626%40DEFAULT'
}

In [47]:
month_filter_toy_girl = {
    '24개월 이하': '1%23attr_10860%242294%7C332717%23attr_11932%2415622%40DEFAULT',
    '2세 이상~5세 이하': '1%23attr_10860%242294%7C332717%23attr_11932%2415623%40DEFAULT',
    '5세 이상~8세 이하': '1%23attr_10860%242294%7C332717%23attr_11932%2415624%40DEFAULT',
    '8세 이상~13세 이하': '1%23attr_10860%242294%7C332717%23attr_11932%2415625%40DEFAULT',
    '13세 이상': '1%23attr_10860%242294%7C332717%23attr_11932%2415626%40DEFAULT'
}

In [48]:
month_filter_toy_uni = {
    '24개월 이하': '1%23attr_10860%242295%7C332717%23attr_11932%2415622%40DEFAULT',
    '2세 이상~5세 이하': '1%23attr_10860%242295%7C332717%23attr_11932%2415623%40DEFAULT',
    '5세 이상~8세 이하': '1%23attr_10860%242295%7C332717%23attr_11932%2415624%40DEFAULT',
    '8세 이상~13세 이하': '1%23attr_10860%242295%7C332717%23attr_11932%2415625%40DEFAULT',
    '13세 이상': '1%23attr_10860%242295%7C332717%23attr_11932%2415626%40DEFAULT'
}

In [49]:
monthList_toy = ['24개월 이하', '2세 이상~5세 이하', '5세 이상~8세 이하', '8세 이상~13세 이하', '13세 이상']

In [50]:
params_toy = {
    "listSize": "60",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "349557",
    "rating": "0",
}

In [51]:
seed_toy = 'http://www.coupang.com/np/categories/349657'

In [69]:
result_toy_boy = {
    monthList_toy[0]: None,
    monthList_toy[1]: None,
    monthList_toy[2]: None,
    monthList_toy[3]: None,
    monthList_toy[4]: None
}

for i in range(5):
    currentpage = 1
    params_toy['filter'] = month_filter_toy_boy[monthList_toy[i]]
    temp = []
    
    while True:
        params_toy['page'] = str(currentpage)
        print(params_toy['filter'])
        if len(getItemNum(seed_toy, params_toy)) == 0:
            break
        print(params_toy['page'])
        temp.extend(getURL(seed_toy, params_toy, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_toy_boy[monthList_toy[i]] = temp

1%23attr_10860%242293%7C332717%23attr_11932%2415622%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242293%7C332717%23attr_11932%2415622%40DEFAULT&component=349557&rating=0
1
1%23attr_10860%242293%7C332717%23attr_11932%2415622%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242293%7C332717%23attr_11932%2415622%40DEFAULT&component=349557&rating=0
2
1%23attr_10860%242293%7C332717%23attr_11932%2415622%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1

5
1%23attr_10860%242293%7C332717%23attr_11932%2415623%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=6&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242293%7C332717%23attr_11932%2415623%40DEFAULT&component=349557&rating=0
6
1%23attr_10860%242293%7C332717%23attr_11932%2415623%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=7&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242293%7C332717%23attr_11932%2415623%40DEFAULT&component=349557&rating=0
7
1%23attr_10860%242293%7C332717%23attr_11932%2415623%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=8&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter

10
1%23attr_10860%242293%7C332717%23attr_11932%2415624%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242293%7C332717%23attr_11932%2415624%40DEFAULT&component=349557&rating=0
11
1%23attr_10860%242293%7C332717%23attr_11932%2415624%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242293%7C332717%23attr_11932%2415624%40DEFAULT&component=349557&rating=0
12
1%23attr_10860%242293%7C332717%23attr_11932%2415624%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=13&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&

15
1%23attr_10860%242293%7C332717%23attr_11932%2415625%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=16&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242293%7C332717%23attr_11932%2415625%40DEFAULT&component=349557&rating=0
16
1%23attr_10860%242293%7C332717%23attr_11932%2415625%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=17&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242293%7C332717%23attr_11932%2415625%40DEFAULT&component=349557&rating=0
17
1%23attr_10860%242293%7C332717%23attr_11932%2415625%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=18&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&

In [70]:
for (k, v) in result_toy_boy.items():
    print(k, len(v))

24개월 이하 1020
2세 이상~5세 이하 1020
5세 이상~8세 이하 1020
8세 이상~13세 이하 1020
13세 이상 1020


In [71]:
result_toy_girl = {
    monthList_toy[0]: None,
    monthList_toy[1]: None,
    monthList_toy[2]: None,
    monthList_toy[3]: None,
    monthList_toy[4]: None
}

for i in range(5):
    currentpage = 1
    params_toy['filter'] = month_filter_toy_girl[monthList_toy[i]]
    temp = []
    
    while True:
        params_toy['page'] = str(currentpage)
        print(params_toy['filter'])
        if len(getItemNum(seed_toy, params_toy)) == 0:
            break
        print(params_toy['page'])
        temp.extend(getURL(seed_toy, params_toy, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_toy_girl[monthList_toy[i]] = temp

1%23attr_10860%242294%7C332717%23attr_11932%2415622%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242294%7C332717%23attr_11932%2415622%40DEFAULT&component=349557&rating=0
1
1%23attr_10860%242294%7C332717%23attr_11932%2415622%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242294%7C332717%23attr_11932%2415622%40DEFAULT&component=349557&rating=0
2
1%23attr_10860%242294%7C332717%23attr_11932%2415622%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1

5
1%23attr_10860%242294%7C332717%23attr_11932%2415623%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=6&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242294%7C332717%23attr_11932%2415623%40DEFAULT&component=349557&rating=0
6
1%23attr_10860%242294%7C332717%23attr_11932%2415623%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=7&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242294%7C332717%23attr_11932%2415623%40DEFAULT&component=349557&rating=0
7
1%23attr_10860%242294%7C332717%23attr_11932%2415623%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=8&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter

10
1%23attr_10860%242294%7C332717%23attr_11932%2415624%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242294%7C332717%23attr_11932%2415624%40DEFAULT&component=349557&rating=0
11
1%23attr_10860%242294%7C332717%23attr_11932%2415624%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242294%7C332717%23attr_11932%2415624%40DEFAULT&component=349557&rating=0
12
1%23attr_10860%242294%7C332717%23attr_11932%2415624%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=13&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&

15
1%23attr_10860%242294%7C332717%23attr_11932%2415625%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=16&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242294%7C332717%23attr_11932%2415625%40DEFAULT&component=349557&rating=0
16
1%23attr_10860%242294%7C332717%23attr_11932%2415625%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=17&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242294%7C332717%23attr_11932%2415625%40DEFAULT&component=349557&rating=0
17
1%23attr_10860%242294%7C332717%23attr_11932%2415625%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=18&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&

In [72]:
for (k, v) in result_toy_girl.items():
    print(k, len(v))

24개월 이하 1020
2세 이상~5세 이하 1020
5세 이상~8세 이하 1020
8세 이상~13세 이하 1020
13세 이상 1020


In [73]:
result_toy_uni = {
    monthList_toy[0]: None,
    monthList_toy[1]: None,
    monthList_toy[2]: None,
    monthList_toy[3]: None,
    monthList_toy[4]: None
}

for i in range(5):
    currentpage = 1
    params_toy['filter'] = month_filter_toy_uni[monthList_toy[i]]
    temp = []
    
    while True:
        params_toy['page'] = str(currentpage)
        print(params_toy['filter'])
        if len(getItemNum(seed_toy, params_toy)) == 0:
            break
        print(params_toy['page'])
        temp.extend(getURL(seed_toy, params_toy, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_toy_uni[monthList_toy[i]] = temp

1%23attr_10860%242295%7C332717%23attr_11932%2415622%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242295%7C332717%23attr_11932%2415622%40DEFAULT&component=349557&rating=0
1
1%23attr_10860%242295%7C332717%23attr_11932%2415622%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242295%7C332717%23attr_11932%2415622%40DEFAULT&component=349557&rating=0
2
1%23attr_10860%242295%7C332717%23attr_11932%2415622%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1

5
1%23attr_10860%242295%7C332717%23attr_11932%2415623%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=6&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242295%7C332717%23attr_11932%2415623%40DEFAULT&component=349557&rating=0
6
1%23attr_10860%242295%7C332717%23attr_11932%2415623%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=7&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242295%7C332717%23attr_11932%2415623%40DEFAULT&component=349557&rating=0
7
1%23attr_10860%242295%7C332717%23attr_11932%2415623%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=8&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter

10
1%23attr_10860%242295%7C332717%23attr_11932%2415624%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=11&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242295%7C332717%23attr_11932%2415624%40DEFAULT&component=349557&rating=0
11
1%23attr_10860%242295%7C332717%23attr_11932%2415624%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=12&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242295%7C332717%23attr_11932%2415624%40DEFAULT&component=349557&rating=0
12
1%23attr_10860%242295%7C332717%23attr_11932%2415624%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=13&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&

15
1%23attr_10860%242295%7C332717%23attr_11932%2415625%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=16&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242295%7C332717%23attr_11932%2415625%40DEFAULT&component=349557&rating=0
16
1%23attr_10860%242295%7C332717%23attr_11932%2415625%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=17&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_10860%242295%7C332717%23attr_11932%2415625%40DEFAULT&component=349557&rating=0
17
1%23attr_10860%242295%7C332717%23attr_11932%2415625%40DEFAULT
http://www.coupang.com/np/categories/349657?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=18&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&

In [74]:
for (k, v) in result_toy_uni.items():
    print(k, len(v))

24개월 이하 1020
2세 이상~5세 이하 1020
5세 이상~8세 이하 1020
8세 이상~13세 이하 1020
13세 이상 1020


In [75]:
toyList = []

In [76]:
def getInfo_toy(url, monthInfo, gender, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_toy(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0 and len(dom.select('span.total-price strong')) > 0:
        name = dom.select('h2.prod-buy-header__title')[0].get_text()
        price = int(dom.select('span.total-price strong')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        url = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        toyList.append(tuple([name, gender, price, url, img, monthInfo]))
    
    return None

In [77]:
import time

def getall_toy_boy():
    startTime = time.time()
    for i in range(5):
        for url in result_toy_boy[monthList_toy[i]]:
            getInfo_toy(url, monthList_toy[i], gender_list[1])
    
    print('Done-boy')
    print(time.time() - startTime)
    return None

def getall_toy_girl():
    startTime = time.time()
    for i in range(5):
        for url in result_toy_girl[monthList_toy[i]]:
            getInfo_toy(url, monthList_toy[i], gender_list[0])
    
    print('Done-girl')
    print(time.time() - startTime)
    return None

def getall_toy_uni():
    startTime = time.time()
    for i in range(5):
        for url in result_toy_uni[monthList_toy[i]]:
            getInfo_toy(url, monthList_toy[i], gender_list[2])
    
    print('Done-uni')
    print(time.time() - startTime)
    return None

In [107]:
import threading
 
thread_boy = threading.Thread(target=getall_toy_boy)
thread_girl = threading.Thread(target=getall_toy_girl)
thread_uni = threading.Thread(target=getall_toy_uni)

thread_boy.start()
thread_girl.start()
thread_uni.start()

In [137]:
len(toyList)

14654

Done-girl
754.4243941307068
Done-boy
754.4473698139191
Done-uni
754.573166847229


In [138]:
len(toyList)

15158

___

## 기저귀
1. 제품명
2. 성별
3. 브랜드
4. 사이즈
5. 가격
6. url
7. img

In [89]:
size_filter_diaper_boy = {
    '신생아': '1%23attr_11880%244488attr_41%242%40DEFAULT',
    '소형': '1%23attr_11880%244489attr_41%242%40DEFAULT',
    '중형': '1%23attr_11880%244490attr_41%242%40DEFAULT',
    '대형': '1%23attr_11880%244491attr_41%242%40DEFAULT',
    '특대': '1%23attr_11880%244492attr_41%242%40DEFAULT',
    '점보': '1%23attr_11880%244493attr_41%242%40DEFAULT'
}

In [90]:
size_filter_diaper_girl = {
    '신생아': '1%23attr_11880%244488attr_41%241%40DEFAULT',
    '소형': '1%23attr_11880%244489attr_41%241%40DEFAULT',
    '중형': '1%23attr_11880%244490attr_41%241%40DEFAULT',
    '대형': '1%23attr_11880%244491attr_41%241%40DEFAULT',
    '특대': '1%23attr_11880%244492attr_41%241%40DEFAULT',
    '점보': '1%23attr_11880%244493attr_41%241%40DEFAULT'
}

In [91]:
size_filter_diaper_uni = {
    '신생아': '1%23attr_11880%244488attr_41%246%40DEFAULT',
    '소형': '1%23attr_11880%244489attr_41%246%40DEFAULT',
    '중형': '1%23attr_11880%244490attr_41%246%40DEFAULT',
    '대형': '1%23attr_11880%244491attr_41%246%40DEFAULT',
    '특대': '1%23attr_11880%244492attr_41%246%40DEFAULT',
    '점보': '1%23attr_11880%244493attr_41%246%40DEFAULT'
}

In [92]:
sizeList_diaper = ['신생아', '소형', '중형', '대형', '특대', '점보']

In [93]:
params_diaper = {
    "listSize": "60",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "221863",
    "rating": "0",
}

In [94]:
seed_diaper = 'http://www.coupang.com/np/categories/221963'

In [95]:
result_diaper_boy = {
    sizeList_diaper[0]: None,
    sizeList_diaper[1]: None,
    sizeList_diaper[2]: None,
    sizeList_diaper[3]: None,
    sizeList_diaper[4]: None,
    sizeList_diaper[5]: None
}

for i in range(6):
    currentpage = 1
    params_diaper['filter'] = size_filter_diaper_boy[sizeList_diaper[i]]
    temp = []
    
    while True:
        params_diaper['page'] = str(currentpage)
        print(params_diaper['filter'])
        if len(getItemNum(seed_diaper, params_diaper)) == 0:
            break
        print(params_diaper['page'])
        temp.extend(getURL(seed_diaper, params_diaper, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_diaper_boy[sizeList_diaper[i]] = temp

1%23attr_11880%244488attr_41%242%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488attr_41%242%40DEFAULT&component=221863&rating=0
1
1%23attr_11880%244488attr_41%242%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488attr_41%242%40DEFAULT&component=221863&rating=0
1%23attr_11880%244489attr_41%242%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244489attr_41%242%40DEFAULT&component=221863&rating=0
1
1%23attr_11880%244489attr_4

In [96]:
for (k, v) in result_diaper_boy.items():
    print(k, len(v))

신생아 60
소형 120
중형 180
대형 420
특대 300
점보 120


In [97]:
result_diaper_girl = {
    sizeList_diaper[0]: None,
    sizeList_diaper[1]: None,
    sizeList_diaper[2]: None,
    sizeList_diaper[3]: None,
    sizeList_diaper[4]: None,
    sizeList_diaper[5]: None
}

for i in range(6):
    currentpage = 1
    params_diaper['filter'] = size_filter_diaper_girl[sizeList_diaper[i]]
    temp = []
    
    while True:
        params_diaper['page'] = str(currentpage)
        print(params_diaper['filter'])
        if len(getItemNum(seed_diaper, params_diaper)) == 0:
            break
        print(params_diaper['page'])
        temp.extend(getURL(seed_diaper, params_diaper, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_diaper_girl[sizeList_diaper[i]] = temp

1%23attr_11880%244488attr_41%241%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488attr_41%241%40DEFAULT&component=221863&rating=0
1
1%23attr_11880%244488attr_41%241%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488attr_41%241%40DEFAULT&component=221863&rating=0
1%23attr_11880%244489attr_41%241%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244489attr_41%241%40DEFAULT&component=221863&rating=0
1
1%23attr_11880%244489attr_4

In [98]:
for (k, v) in result_diaper_girl.items():
    print(k, len(v))

신생아 60
소형 120
중형 180
대형 360
특대 360
점보 120


In [99]:
result_diaper_uni = {
    sizeList_diaper[0]: None,
    sizeList_diaper[1]: None,
    sizeList_diaper[2]: None,
    sizeList_diaper[3]: None,
    sizeList_diaper[4]: None,
    sizeList_diaper[5]: None
}

for i in range(6):
    currentpage = 1
    params_diaper['filter'] = size_filter_diaper_uni[sizeList_diaper[i]]
    temp = []
    
    while True:
        params_diaper['page'] = str(currentpage)
        print(params_diaper['filter'])
        if len(getItemNum(seed_diaper, params_diaper)) == 0:
            break
        print(params_diaper['page'])
        temp.extend(getURL(seed_diaper, params_diaper, select='ul.baby-product-list a.baby-product-link'))
        currentpage += 1
        
    result_diaper_uni[sizeList_diaper[i]] = temp

1%23attr_11880%244488attr_41%246%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488attr_41%246%40DEFAULT&component=221863&rating=0
1
1%23attr_11880%244488attr_41%246%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488attr_41%246%40DEFAULT&component=221863&rating=0
2
1%23attr_11880%244488attr_41%246%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488attr_41%246%40DEFAULT&component=221863&rating=0
3
1%23attr_11880%244488attr

1
1%23attr_11880%244491attr_41%246%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=2&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244491attr_41%246%40DEFAULT&component=221863&rating=0
2
1%23attr_11880%244491attr_41%246%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=3&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244491attr_41%246%40DEFAULT&component=221863&rating=0
3
1%23attr_11880%244491attr_41%246%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=4&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244491attr_41%246%40DEFAULT&component=221863&rating=0
4
1%23attr_11880%244491at

3
1%23attr_11880%244493attr_41%246%40DEFAULT
http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=4&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244493attr_41%246%40DEFAULT&component=221863&rating=0


In [100]:
for (k, v) in result_diaper_uni.items():
    print(k, len(v))

신생아 360
소형 420
중형 540
대형 720
특대 600
점보 180


In [202]:
diaperList = []

In [102]:
def getInfo_diaper(url, sizeInfo, gender, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_diaper(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0 and len(dom.select('span.total-price strong')) > 0:
        name = dom.select('h2.prod-buy-header__title')[0].get_text()
        brand = dom.select('a.prod-brand-name')[0].get_text()
        price = int(dom.select('span.total-price strong')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        url = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        diaperList.append(tuple([name, gender, brand, sizeInfo, price, url, img]))
    
    return None

In [163]:
def getall_diaper_boy():
    startTime = time.time()
    for i in range(6):
        for url in result_diaper_boy[sizeList_diaper[i]]:
            getInfo_diaper(url, sizeList_diaper[i], gender_list[1])
    
    print('Done-boy')
    print(time.time() - startTime)
    return None

def getall_diaper_girl():
    startTime = time.time()
    for i in range(6):
        for url in result_diaper_girl[sizeList_diaper[i]]:
            getInfo_diaper(url, sizeList_diaper[i], gender_list[0])
    
    print('Done-girl')
    print(time.time() - startTime)
    return None

def getall_diaper_uni():
    startTime = time.time()
    for i in range(6):
        for url in result_diaper_uni[sizeList_diaper[i]]:
            getInfo_diaper(url, sizeList_diaper[i], gender_list[2])
    
    print('Done-uni')
    print(time.time() - startTime)
    return None

In [203]:
import threading
 
thread_boy = threading.Thread(target=getall_diaper_boy)
thread_girl = threading.Thread(target=getall_diaper_girl)
thread_uni = threading.Thread(target=getall_diaper_uni)

thread_boy.start()
thread_girl.start()
thread_uni.start()

In [227]:
import sqlite3 as sql

In [230]:
con = sql.connect('coupang-bulk.db')

with con:
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS Snack")
    cur.execute("CREATE TABLE Snack(Brand TEXT, Name TEXT, Price INT, url TEXT, img TEXT, Monthinfo TEXT)")
    cur.executemany("INSERT INTO Snack VALUES (?, ?, ?, ?, ?, ?)", snackList)
    
    cur.execute("DROP TABLE IF EXISTS Milkpowder")
    cur.execute("CREATE TABLE Milkpowder(Brand TEXT, Name TEXT, Price INT, url TEXT, img TEXT, Monthinfo TEXT)")
    cur.executemany("INSERT INTO Milkpowder VALUES (?, ?, ?, ?, ?, ?)", milkPowderList)
    
    cur.execute("DROP TABLE IF EXISTS Clothes")
    cur.execute("CREATE TABLE Clothes(Name TEXT, Gender TEXT, Price INT, url TEXT, img TEXT, Monthinfo TEXT)")
    cur.executemany("INSERT INTO Clothes VALUES (?, ?, ?, ?, ?, ?)", clothesList)
    
    cur.execute("DROP TABLE IF EXISTS Toy")
    cur.execute("CREATE TABLE Toy(Name TEXT, Gender TEXT, Price INT, url TEXT, img TEXT, Monthinfo TEXT)")
    cur.executemany("INSERT INTO Toy VALUES (?, ?, ?, ?, ?, ?)", toyList)
    
    cur.execute("DROP TABLE IF EXISTS Diaper")
    cur.execute("CREATE TABLE Diaper(Name TEXT, Gender TEXT, Brand TEXT, Sizeinfo TEXT, Price INT, url TEXT, img TEXT)")
    cur.executemany("INSERT INTO Diaper VALUES (?, ?, ?, ?, ?, ?, ?)", diaperList)